<a href="https://colab.research.google.com/github/DarkT2014/Model_experiments_colab/blob/main/Translation_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!pip install datasets transformers[sentencepiece]
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, Trainer, TrainingArguments
from datasets import load_dataset

# 加载预训练模型和分词器
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# 加载你的翻译数据
dataset = load_dataset("un_multi", "en-zh")
print("Data loaded.")





config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split:   0%|          | 0/9564315 [00:00<?, ? examples/s]

Data loaded.


In [ ]:
# 假设你要从数据集中切分出10%的数据
full_dataset = dataset['train']
# 因为这里源数据已经划分了train，这里只是想选择train其中的一小部分验证代码的可行性。

small_dataset = full_dataset.train_test_split(test_size=0.001)['test']
train_dataset = full_dataset.train_test_split(test_size=0.009)['test']

# small_dataset现在有两个部分：'train'（90%的数据）和'test'（你的10%小数据集）
# 你可以选择使用'test'部分作为你的小数据集
# small_dataset = dataset['test']

print(small_dataset)
print(train_dataset)



# # -------
# # 或者，如果你想通过指定样本数来切分数据集：
# import random

# # 假设你想从数据集中随机选择1000个样本
# sample_indices = random.sample(range(len(dataset)), 1000)
# small_dataset = dataset.select(sample_indices)




# ---------------

def preprocess_function(examples):
    # 分别提取英文和中文句子
    english_sentences = [ex['en'] for ex in examples['translation']]
    chinese_sentences = [ex['zh'] for ex in examples['translation']]

    # 使用tokenizer进行编码
    # model_inputs = tokenizer(english_sentences, max_length=128, truncation=True)
    model_inputs = tokenizer(english_sentences, max_length=128, truncation=True, padding="max_length")

    # 设置解码器的输入
    with tokenizer.as_target_tokenizer():
        # labels = tokenizer(chinese_sentences, max_length=128, truncation=True)
        labels = tokenizer(chinese_sentences, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 应用预处理
# tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = small_dataset.map(preprocess_function, batched=True)


# small_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
# small_val_dataset = small_val_dataset.map(preprocess_function, batched=True)




Dataset({
    features: ['translation'],
    num_rows: 9565
})
Dataset({
    features: ['translation'],
    num_rows: 86079
})


Map:   0%|          | 0/86079 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9565 [00:00<?, ? examples/s]

转换为TensorFlow数据集

将数据集转换为TensorFlow的格式：

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


train_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)
eval_dataset = tokenized_test_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4. 微调模型

接下来，配置模型的训练过程。首先，定义优化器、损失函数和评价指标：

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=5e-5)

model.compile(
    optimizer=optimizer,
    loss={"loss": None},
    metrics=["accuracy"]
)


TypeError: ignored

然后，开始训练：

In [ ]:
model.fit(train_dataset, validation_data=eval_dataset, epochs=3)


5. 评估和保存模型

在微调后，使用验证集评估模型性能：

In [ ]:
model.evaluate(eval_dataset)


如果对模型性能满意，可以将其保存起来：

In [ ]:
model.save_pretrained("my_finetuned_model")






















这里是一个简单的示例，展示了如何在 T5 模型的基础上添加一个额外的线性层来实现模型结构的修改

In [ ]:
import torch
from torch import nn
from transformers import T5ForConditionalGeneration, T5Config

class CustomT5Model(T5ForConditionalGeneration):
    def __init__(self, config):
        super(CustomT5Model, self).__init__(config)
        self.additional_linear = nn.Linear(config.d_model, config.d_model)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        encoder_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        decoder_inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        **kwargs
    ):
        outputs = super().forward(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            encoder_outputs=encoder_outputs,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            labels=labels,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            **kwargs
        )

        # Apply the additional linear layer to the last hidden state of the decoder
        additional_output = self.additional_linear(outputs.last_hidden_state)
        return outputs, additional_output

# Load the T5 configuration and initialize the custom model
config = T5Config.from_pretrained('t5-small')
custom_model = CustomT5Model(config)
